In [14]:
# importando as bibliotecas

import time
import pandas as pd
import pyodbc
from urllib.error import URLError, HTTPError
from urllib.request import Request, urlopen

import bs4

In [15]:
# informando o agent

user_agent = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36 Edg/110.0.1587.63'
headers = {'User-Agent': user_agent}

In [16]:
# função para chamar a página WEB
def ConsultaWebB(url):
    try:
        req = Request(url, headers = headers)
        response = urlopen(req)
        return response.read()
    except:
        pass
    
# função para pegar a página e transformar em um objeto
def captura_html_pagina(url):
    html = ConsultaWebB(url)
    soup = bs4.BeautifulSoup(html, 'html.parser')
    return soup

# função para capturar as informações da "tabelinha cabeçalho da página"
def Cabecalho(html):
    dt = html.findAll('dt')
    dd = html.findAll('dd')
    dic = {}
    for i in range(len(dt)):
        x = dt[i].get_text()
        y = dd[i].get_text()
        dic[x] = y
    return dic

# função para capturar o conteúdo da página
def Conteudo(proposicao, ano):
    url = 'https://www.legislador.com.br//LegisladorWEB.ASP?WCI=ProposicaoTexto&ID=3&TPProposicao=1&nrProposicao='+str(proposicao)+'&aaProposicao=' + str(ano)
    html = captura_html_pagina(url)
    dic = Cabecalho(html)
    dic['Proposição'] = proposicao
    dic['Ano'] = ano
    dic['Texto'] = html.p.get_text()
    return dic

def TabelaResultados(iniciar_em, quantidade, ano, erros_admissiveis, segundos_espera):
    ultima_consulta  = iniciar_em + quantidade - 1
    
    erros = 0
    i = 1
    lista = []
    
    while iniciar_em <= ultima_consulta and erros <= erros_admissiveis:
        
        try:
            x = Conteudo(iniciar_em,ano)
            lista += [Conteudo(iniciar_em, ano)]
        except:
            erros += 1
            pass
        
        time.sleep(segundos_espera)
        
        iniciar_em += 1
        i+= 1
    return pd.DataFrame(lista)

In [17]:
teste = TabelaResultados(401,4,2021,2,0.2)

In [18]:
teste

,Reunião,Deliberação,Situação,Assunto,Autor,Proposição,Ano,Texto
0,15/03/2021,15/03/2021,Proposição Aprovada,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Roger Michel Knipers.,401,2021,"O vereador abaixo firmado requer, após ouvido ..."
1,15/03/2021,15/03/2021,Proposição Aprovada,Manutenção de via pública,Vereador Ana Paula Reiter.,402,2021,"A vereadora abaixo firmada requer, após ouvido..."
2,18/03/2021,NaN,Entrada na Ordem do Dia,Manutenção de via pública,Vereador Jonas Luiz de Lima.,403,2021,"O Vereador abaixo firmado requer, após ouvido ..."
3,18/03/2021,NaN,Entrada na Ordem do Dia,"Limpeza, Macadamização, Patrolamento, Retifica...",Vereador Flávio Augusto Ferri Molinari.,404,2021,"O vereador que está subscreve, no uso das atri..."


In [19]:
teste.columns

Index(['Reunião', 'Deliberação', 'Situação', 'Assunto', 'Autor', 'Proposição',
       'Ano', 'Texto'],
      dtype='object')

In [20]:
teste.dtypes

Reunião        object
Deliberação    object
Situação       object
Assunto        object
Autor          object
Proposição      int64
Ano             int64
Texto          object
dtype: object

# Inserindo na base de dados com SQL com atualização incremental